In [1]:
import os
import xarray as xr

model_init_date = "2025121300"
fpath = '/data/projects/derived_products/ECMWF_IVT/Ensemble/'
fname = os.path.join(fpath, f'IVT_EC_{model_init_date}.nc')
ds=xr.open_dataset(fname)
ds

ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/SEAK-impacts/share/proj failed
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'gini' loading failed:
Struct() takes at most 1 argument (3 given)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:         (ensemble: 51, forecast_time: 121, lat: 121, lon: 291,
                     ncl4: 121)
Coordinates:
  * ensemble        (ensemble) int32 0 1 2 3 4 5 6 7 ... 43 44 45 46 47 48 49 50
  * lat             (lat) float32 10.0 10.5 11.0 11.5 ... 68.5 69.0 69.5 70.0
  * lon             (lon) float32 150.0 150.5 151.0 151.5 ... 294.0 294.5 295.0
Dimensions without coordinates: forecast_time, ncl4
Data variables:
    IVT             (ensemble, forecast_time, lat, lon) float32 ...
    uIVT            (ensemble, forecast_time, lat, lon) float32 ...
    vIVT            (ensemble, forecast_time, lat, lon) float32 ...
    forecast_times  (ncl4) int32 ...

In [2]:
ds.lon.values

array([150. , 150.5, 151. , 151.5, 152. , 152.5, 153. , 153.5, 154. ,
       154.5, 155. , 155.5, 156. , 156.5, 157. , 157.5, 158. , 158.5,
       159. , 159.5, 160. , 160.5, 161. , 161.5, 162. , 162.5, 163. ,
       163.5, 164. , 164.5, 165. , 165.5, 166. , 166.5, 167. , 167.5,
       168. , 168.5, 169. , 169.5, 170. , 170.5, 171. , 171.5, 172. ,
       172.5, 173. , 173.5, 174. , 174.5, 175. , 175.5, 176. , 176.5,
       177. , 177.5, 178. , 178.5, 179. , 179.5, 180. , 180.5, 181. ,
       181.5, 182. , 182.5, 183. , 183.5, 184. , 184.5, 185. , 185.5,
       186. , 186.5, 187. , 187.5, 188. , 188.5, 189. , 189.5, 190. ,
       190.5, 191. , 191.5, 192. , 192.5, 193. , 193.5, 194. , 194.5,
       195. , 195.5, 196. , 196.5, 197. , 197.5, 198. , 198.5, 199. ,
       199.5, 200. , 200.5, 201. , 201.5, 202. , 202.5, 203. , 203.5,
       204. , 204.5, 205. , 205.5, 206. , 206.5, 207. , 207.5, 208. ,
       208.5, 209. , 209.5, 210. , 210.5, 211. , 211.5, 212. , 212.5,
       213. , 213.5,

In [3]:
def clean_ds(ds, forecast):
    # normalize longitudes to [-180, 180)
    if 'lon' in ds.coords or 'longitude' in ds.coords:
        # prefer 'lon' coordinate name but support both
        lon_name = 'lon' if 'lon' in ds.coords else 'longitude'
        ds = ds.assign_coords({lon_name: ((ds[lon_name] + 180) % 360 - 180)}).sortby(lon_name)
        # if needed, rename to 'lon'/'lat' canonical names
        if 'longitude' in ds.coords and 'lon' not in ds.coords:
            ds = ds.rename({'longitude': 'lon'})
        if 'latitude' in ds.coords and 'lat' not in ds.coords:
            ds = ds.rename({'latitude': 'lat'})

    # Model-specific adjustments
    if forecast == 'ECMWF':
        if 'forecast_time' in ds.dims:
            ds = ds.rename_dims({'forecast_time': 'forecast_hour'})

        if 'forecast_times' in ds:
            ds = ds.assign_coords(
                forecast_hour=ds['forecast_times'].values
            )
            ds = ds.drop_vars('forecast_times')
        # historic: for years > 2020 forecast_hour was stored differently
        try:
            dt_init = datetime.datetime.strptime(self.model_init_date, "%Y%m%d%H")
            if int(dt_init.strftime('%Y')) > 2020 and 'forecast_hour' in ds:
                ds['forecast_hour'] = ds['forecast_hour'] * 3
        except Exception:
            pass
        
    return ds

In [4]:
ds = clean_ds(ds, "ECMWF")
ds

<xarray.Dataset>
Dimensions:        (ensemble: 51, forecast_hour: 121, lat: 121, lon: 291)
Coordinates:
  * ensemble       (ensemble) int32 0 1 2 3 4 5 6 7 ... 43 44 45 46 47 48 49 50
  * lat            (lat) float32 10.0 10.5 11.0 11.5 ... 68.5 69.0 69.5 70.0
  * lon            (lon) float32 -180.0 -179.5 -179.0 ... 178.5 179.0 179.5
  * forecast_hour  (forecast_hour) int32 0 3 6 9 12 15 ... 348 351 354 357 360
Data variables:
    IVT            (ensemble, forecast_hour, lat, lon) float32 ...
    uIVT           (ensemble, forecast_hour, lat, lon) float32 ...
    vIVT           (ensemble, forecast_hour, lat, lon) float32 ...

In [8]:
ds = ds.sel(lon=slice(-170, -105))
ds

<xarray.Dataset>
Dimensions:        (ensemble: 51, forecast_hour: 121, lat: 121, lon: 131)
Coordinates:
  * ensemble       (ensemble) int32 0 1 2 3 4 5 6 7 ... 43 44 45 46 47 48 49 50
  * lat            (lat) float32 10.0 10.5 11.0 11.5 ... 68.5 69.0 69.5 70.0
  * lon            (lon) float32 -170.0 -169.5 -169.0 ... -106.0 -105.5 -105.0
  * forecast_hour  (forecast_hour) int32 0 3 6 9 12 15 ... 348 351 354 357 360
Data variables:
    IVT            (ensemble, forecast_hour, lat, lon) float32 ...
    uIVT           (ensemble, forecast_hour, lat, lon) float32 ...
    vIVT           (ensemble, forecast_hour, lat, lon) float32 ...

In [9]:
fpath = '/data/projects/derived_products/GEFS_IVT/data/'
fname = os.path.join(fpath, f'GEFS_IVT_{model_init_date}.nc')
ds2 = xr.open_dataset(fname)
ds2 = clean_ds(ds2, "GEFS")
ds2 = ds2.sel(lat=slice(70, 10), lon=slice(-170, -105))
ds2

<xarray.Dataset>
Dimensions:        (forecast_hour: 105, ensemble: 31, lat: 121, lon: 131)
Coordinates:
  * ensemble       (ensemble) int32 0 1 2 3 4 5 6 7 ... 23 24 25 26 27 28 29 30
  * forecast_hour  (forecast_hour) int32 0 3 6 9 12 15 ... 360 366 372 378 384
  * lat            (lat) float32 70.0 69.5 69.0 68.5 ... 11.5 11.0 10.5 10.0
  * lon            (lon) float32 -170.0 -169.5 -169.0 ... -106.0 -105.5 -105.0
Data variables:
    IVT            (forecast_hour, ensemble, lat, lon) float32 ...
    uIVT           (forecast_hour, ensemble, lat, lon) float32 ...
    vIVT           (forecast_hour, ensemble, lat, lon) float32 ...

In [7]:
ds2.lon.values

array([-180. , -179.5, -179. , -178.5, -178. , -177.5, -177. , -176.5,
       -176. , -175.5, -175. , -174.5, -174. , -173.5, -173. , -172.5,
       -172. , -171.5, -171. , -170.5, -170. , -169.5, -169. , -168.5,
       -168. , -167.5, -167. , -166.5, -166. , -165.5, -165. , -164.5,
       -164. , -163.5, -163. , -162.5, -162. , -161.5, -161. , -160.5,
       -160. , -159.5, -159. , -158.5, -158. , -157.5, -157. , -156.5,
       -156. , -155.5, -155. , -154.5, -154. , -153.5, -153. , -152.5,
       -152. , -151.5, -151. , -150.5, -150. , -149.5, -149. , -148.5,
       -148. , -147.5, -147. , -146.5, -146. , -145.5, -145. , -144.5,
       -144. , -143.5, -143. , -142.5, -142. , -141.5, -141. , -140.5,
       -140. , -139.5, -139. , -138.5, -138. , -137.5, -137. , -136.5,
       -136. , -135.5, -135. , -134.5, -134. , -133.5, -133. , -132.5,
       -132. , -131.5, -131. , -130.5, -130. , -129.5, -129. , -128.5,
       -128. , -127.5, -127. , -126.5, -126. , -125.5, -125. , -124.5,
      

In [11]:
fpath = '/data/downloaded/WWRF-NRT/2025-2026/Ensemble_IVT/'
fname = os.path.join(fpath, f'IVT_WWRF_{model_init_date}.nc')
ds3 = xr.open_dataset(fname)
ds3

<xarray.Dataset>
Dimensions:        (ensembles: 200, forecast_hour: 57, lat: 111, lon: 131,
                    ncl4: 200)
Coordinates:
  * lat            (lat) float64 10.0 10.5 11.0 11.5 ... 63.5 64.0 64.5 65.0
  * lon            (lon) float64 185.0 185.5 186.0 186.5 ... 249.0 249.5 250.0
    ensembles      (ncl4) float64 ...
  * forecast_hour  (forecast_hour) int32 0 3 6 9 12 15 ... 156 159 162 165 168
Dimensions without coordinates: ncl4
Data variables:
    IVT            (ensembles, forecast_hour, lat, lon) float32 ...
    uIVT           (ensembles, forecast_hour, lat, lon) float32 ...
    vIVT           (ensembles, forecast_hour, lat, lon) float32 ...
Attributes:
    history:                    Sat Dec 13 08:34:28 2025: /home/cw3eprod/mini...
    NCO:                        netCDF Operators version 5.2.9 (Homepage = ht...
    history_of_appended_files:  Sat Dec 13 08:34:28 2025: Appended file /cw3e...

In [12]:
ds3 = clean_ds(ds3, "W-WRF")
ds3

<xarray.Dataset>
Dimensions:        (ensembles: 200, forecast_hour: 57, lat: 111, lon: 131,
                    ncl4: 200)
Coordinates:
  * lat            (lat) float64 10.0 10.5 11.0 11.5 ... 63.5 64.0 64.5 65.0
    ensembles      (ncl4) float64 ...
  * forecast_hour  (forecast_hour) int32 0 3 6 9 12 15 ... 156 159 162 165 168
  * lon            (lon) float64 -175.0 -174.5 -174.0 ... -111.0 -110.5 -110.0
Dimensions without coordinates: ncl4
Data variables:
    IVT            (ensembles, forecast_hour, lat, lon) float32 ...
    uIVT           (ensembles, forecast_hour, lat, lon) float32 ...
    vIVT           (ensembles, forecast_hour, lat, lon) float32 ...
Attributes:
    history:                    Sat Dec 13 08:34:28 2025: /home/cw3eprod/mini...
    NCO:                        netCDF Operators version 5.2.9 (Homepage = ht...
    history_of_appended_files:  Sat Dec 13 08:34:28 2025: Appended file /cw3e...